In [1]:
#%%
import sys
# sys.path.append("/home/baokhanh/")
import pandas as pd
import numpy as np
import os
import calendar
import psycopg2
from pathlib import Path
from loguru import logger
from dotenv import load_dotenv
from datetime import datetime, timedelta
from unidecode import unidecode
from avay_bq import AvayBQServiceAcc

from slack import WebClient
from slack.errors import SlackApiError

import calendar
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse
from math import floor, ceil

from da_utils.client.gmail_client import GmailClient
from da_utils.repository.gmail.gmail_repository import GmailRepository
from da_utils.repository.gmail.custom_types import GmailMessage

from da_utils.client.google_sheets_client import GoogleSheetsClient
from da_utils.repository.google_sheets.google_sheets_repository import GoogleSheetsService

from da_utils.client.drive_client import GoogleDriveClient
from da_utils.repository.drive.drive_repository import GoogleDriveService

from da_utils.client.slack_client import SlackClient
from da_utils.repository.slack.slack_repository import SlackRepository

from da_utils.client.bigquery_client import BigqueryClient
from da_utils.repository.bigquery.bigquery_repository import BigqueryService

load_dotenv()
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 2000

/home/linhnguyen/.local/lib/python3.9/site-packages/slack/deprecation.py:14: UserWarning: slack package is deprecated. Please use slack_sdk.web/webhook/rtm package instead. For more info, go to https://slack.dev/python-slack-sdk/v3-migration/
  warnings.warn(message)
/home/linhnguyen/.local/lib/python3.9/site-packages/slack/deprecation.py:14: UserWarning: slack.errors package is deprecated. Please use slack_sdk.errors package instead. For more info, go to https://slack.dev/python-slack-sdk/v3-migration/
  warnings.warn(message)


In [2]:
gmail_client = GmailClient(Path(os.environ.get("GOOGLE_CREDENTIALS_PATH")), Path(os.environ.get("GMAIL_TOKEN_PATH")))
gmail_repo = GmailRepository(gmail_client)

drive_client = GoogleDriveClient( )
drive_repo = GoogleDriveService(drive_client)

ggsheet_client = GoogleSheetsClient( )
ggsheet_repo = GoogleSheetsService(ggsheet_client)

slack_client = SlackClient(token=os.environ.get("SLACK_BOT_AUTHEN"))
slack_repo = SlackRepository(client=slack_client)

conn = psycopg2.connect(
    host = "v1proxy-new-postgres-replica.cvhqeqp3dmlv.ap-southeast-1.rds.amazonaws.com",
    dbname = "score_api_production",
    user = "analytic",
    password = "COng@tr0ngcuctacuctac",
    port = "5432"
)

bot_auth_token = os.environ.get("SLACK_BOT_AUTHEN")
user_auth_token = os.environ.get("SLACK_USER_AUTHEN")
clientSlack = WebClient(token = bot_auth_token)

channel_id = "C04MYU3L7LN"
user_id = "U0480LX5468"
linh_id = "U047QMYB9TQ"


In [4]:
def dayrange_recon(day_from_curr: int = 45) -> tuple:
    curr_time = datetime.now()
    recon_time =(curr_time - timedelta(day_from_curr))
    last_month_time = (recon_time - timedelta(30))
    last_2month_time = (recon_time - timedelta(60))
    _, end_date = calendar.monthrange(int(recon_time.strftime("%Y")), int(recon_time.strftime("%m")))

    recon_month = recon_time.strftime("%m-%Y")
    last_month = last_month_time.strftime("%Y-%m")
    last_2month = last_2month_time.strftime("%Y-%m")
    month = recon_time.strftime("%Y-%m")
    start_date = recon_time.strftime("%Y-%m") + "-" + "01"
    end_date = recon_time.strftime("%Y-%m") + "-" + str(end_date)
    return recon_month, month, recon_time, start_date, end_date, last_month, last_2month, last_month_time, last_2month_time

recon_month, month, recon_time, start_date, end_date, last_month, last_2month, last_month_time, last_2month_time = dayrange_recon()
recon_month

'08-2023'

In [5]:
def create_or_get_folder(recon_month, parent_folder_id):
    folder_check = drive_repo.list_folder_names(parent_folder_id =  parent_folder_id)
    if recon_month not in folder_check:
        folder_id = drive_repo.create_folder(folder_name= recon_month, parent_folder_id =  parent_folder_id)
    else:
        folder_id = drive_repo.get_folders(folder_name= recon_month, parent_folder_id =  parent_folder_id)
        folder_id = folder_id[0]
    return folder_id

def create_or_get_workbook(parent_folder_id, wb_name):
    wb_check = drive_repo.list_file_names(parent_folder_id= parent_folder_id)
    if wb_name not in wb_check:
        wb = ggsheet_repo.create_spreadsheet(spreadsheet_name = wb_name,parent_folder_id = parent_folder_id)
    else:
        spreadsheet_key = drive_repo.get_files(file_name = wb_name,parent_folder_id = parent_folder_id)
        spreadsheet_key = spreadsheet_key[0]
        wb = ggsheet_repo.open_spreadsheet(spreadsheet_key = spreadsheet_key)
    return wb

def create_or_update_sheet(wb, new_sheet, input_df, starting_cell: str = 'A1'):
    sheet_names = [worksheet.title for worksheet in wb.worksheets()]
    if new_sheet not in sheet_names:
        wb.add_worksheet(title=new_sheet, rows=100, cols=20)
        ws = wb.worksheet(new_sheet)
        ggsheet_repo.write_df_to_sheet(worksheet = ws, df_to_write = input_df, starting_cell=starting_cell)
    else:
        ws = wb.worksheet(new_sheet)
        ws.clear()
        ggsheet_repo.write_df_to_sheet(worksheet = ws, df_to_write = input_df, starting_cell=starting_cell)
    if 'Sheet1' in sheet_names: 
        wb.del_worksheet(wb.worksheet('Sheet1')) 
    else:
        pass
    return

parent_folder_id = '1qvIX-uNfUEgtj99uE0qQgnEsa4Ryx1vH'


In [6]:
folder_month = create_or_get_folder(recon_month = month, parent_folder_id = parent_folder_id)
lotte_check_report =create_or_get_workbook(parent_folder_id = folder_month, wb_name = f'lotte_check_report {month}')
lotte_summary = create_or_get_workbook(parent_folder_id = folder_month, wb_name = f'lotte_summary_{month}')
Lotte_detail_leads_by_sender = create_or_get_workbook(parent_folder_id = folder_month, wb_name = f'Lotte_detail_leads_by_sender_{month}')
lotte_statement = create_or_get_workbook(parent_folder_id = folder_month, wb_name = f'lotte_statement_{month}')
path = '/home/linhnguyen/04.Reconcile/03.Lotte/'

I - GET FILE FROM GMAIL

1.1. MOBIFONE

In [9]:
list_month = [month, last_month,last_2month]
mbf_lead_list = []
for months in list_month:
    QUERY_STRING=f'"[TS-LOTTE-MOBIFONE] Leadgen Services Reconciliation for {months}" has:attachment'

    gmail_messages = gmail_repo.search_messages_by_query(query=QUERY_STRING)
    for message in gmail_messages:
        attachment_pair_id = gmail_repo.query_attachments_from_email_message(message=message)
        print(attachment_pair_id[0].attachment_file_name)
        print(months)
        df = gmail_repo.read_excel_attachment_to_pandas(attachment_pair = attachment_pair_id[0],
                                                   password = 'lfvn2023', sheet_name = 'TS_mbf')
        df['month'] = months
        mbf_lead_list.append(df)
mbf_lead = pd.concat(mbf_lead_list)
mbf_lead

Q_2023.08_TS_mbf_sent.xlsx
2023-08
Q_2307_sent TS_mbf_sms.xlsx
2023-07
Q_2306_sent TS_mbf_sms.xlsx
2023-06


,Q,Reason,ImportDate,Mobile2,Address,Sex,DOB,Age,SourceData,Score,Telco,SubmitData,CIC,DuplicateData,LoanDemand,FailReason,CallBackReason,FollowReason,month
0,N,Follow,2023-08-01 06:00:59,14224015,Hồ Chí Minh,Nữ,NaN,05/05/1995,TrustingSocial,645-649,mobifone,sms,CIC thỏa mãn,Không trùng data,NaN,NaN,NaN,NaN,2023-08
1,N,Follow,2023-08-01 06:00:58,14224012,NaN,NaN,NaN,NaN,TrustingSocial,700-704,mobifone,sms,NaN,NaN,NaN,NaN,NaN,NaN,2023-08
2,N,Nợ xấu,2023-08-01 06:00:58,14224006,Đồng Tháp,NaN,NaN,29/09/1992,TrustingSocial,630-634,mobifone,sms,Nợ xấu,NaN,NaN,08.1 - CIC nợ xấu,NaN,NaN,2023-08
0,N,Ngoài độ tuổi,2023-07-01 06:00:56.317000,14215160,Đà Nẵng,Nam,NaN,01/01/1967,TrustingSocial,685-689,mobifone,sms,CIC trắng,Không trùng data,NaN,08.2 - CIC trắng,NaN,NaN,2023-07
1,N,Hồ sơ chờ thu,2023-07-01 06:00:57.253000,14215183,Long An,Nam,NaN,31/07/1984,TrustingSocial,645-649,mobifone,sms,CIC thỏa mãn,Không trùng data,08 - Có nhu cầu,NaN,NaN,NaN,2023-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2765,N,Call back,2023-06-30 16:30:56.603000,14215042,Đồng Nai,Nữ,NaN,01/01/1996,TrustingSocial,640-644,mobifone,sms,CIC trắng,NaN,NaN,NaN,NaN,NaN,2023-06
2766,N,Nợ chú ý,2023-06-30 16:35:54.620000,14215054,Long An,NaN,NaN,01/01/1978,TrustingSocial,635-639,mobifone,sms,Nợ chú ý,Không trùng data,NaN,NaN,NaN,NaN,2023-06
2767,N,Nợ xấu,2023-06-30 16:40:54.793000,14215063,Đồng Nai,Nam,NaN,01/01/1978,TrustingSocial,650-654,mobifone,sms,Nợ xấu,Không trùng data,NaN,08.1 - CIC nợ xấu,NaN,NaN,2023-06
2768,N,Call back,2023-06-30 16:40:57.123000,14215065,Đà Nẵng,Nam,NaN,02/06/1993,TrustingSocial,670-674,mobifone,sms,CIC thỏa mãn,Không trùng data,09 - Không có nhu cầu: Không muốn vay nữa,NaN,NaN,NaN,2023-06


In [10]:
mbf_lead = mbf_lead.rename(columns = {'Q': 'qualify', 'Mobile2': 'lead_id'})
mbf_lead.groupby(['month', 'Telco', 'SubmitData']).agg({'lead_id': 'count'})

,,,lead_id
month,Telco,SubmitData,
2023-06,mobifone,sms,2770
2023-07,mobifone,sms,1202
2023-08,mobifone,sms,3


In [18]:
QUERY_STRING=f'"[TS-LOTTE-MOBIFONE] Leadgen Services Reconciliation for {month}" has:attachment'

gmail_messages = gmail_repo.search_messages_by_query(query=QUERY_STRING)

for message in gmail_messages:
    attachment_pair_id = gmail_repo.query_attachments_from_email_message(message=message)
    print(attachment_pair_id[0].attachment_file_name)
    mbf_data_disbursed = gmail_repo.read_excel_attachment_to_pandas(attachment_pair = attachment_pair_id[0],
                                                   password = 'lfvn2023', sheet_name = 'E_TS_mbf', skiprows= 9)
mbf_data_disbursed = mbf_data_disbursed.iloc[:, 8:]
mbf_data_disbursed

Q_2023.08_TS_mbf_sent.xlsx


,Application No,LOANAMOUNT,DISBURSE_DATE,Import date,No of days,Fee non Vat,Fee VAT,Lead_ID
0,APL011818576,20000000,2023-08-02 00:00:00,2023-07-22 06:00:55.607000,10.749356400498073,1100000,1210000,14222606
1,APL011821164,40000000,2023-08-04 00:00:00,2022-12-23 00:00:00,224,0,0,13450736
2,APL011825958,50000000,2023-08-14 00:00:00,2023-06-15 16:00:52.883000,59.332721261598635,2750000,3025000.0000000005,14169551
3,APL011828148,50000000,2023-08-18 00:00:00,2023-06-17 06:00:58.677000,61.74932086809713,2750000,3025000.0000000005,14173732
4,APL011832241,30000000,2023-08-23 00:00:00,2022-12-21 00:00:00,245,0,0,13439583


In [19]:
mbf_data_disbursed['No of days'] = mbf_data_disbursed['No of days'].astype(float).round(0).astype(int)
mbf_data_disbursed = mbf_data_disbursed[mbf_data_disbursed['No of days']<= 90]
mbf_data_disbursed.rename(columns = {'Lead_ID':'lead_id', 'PHONE':'phone_number'}, inplace = True)
mbf_data_disbursed.columns = mbf_data_disbursed.columns.str.lower()
mbf_data_disbursed.columns = mbf_data_disbursed.columns.str.replace(' ', '_')
mbf_data_disbursed

/tmp/ipykernel_510939/3117477869.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mbf_data_disbursed.rename(columns = {'Lead_ID':'lead_id', 'PHONE':'phone_number'}, inplace = True)


,application_no,loanamount,disburse_date,import_date,no_of_days,fee_non_vat,fee_vat,lead_id
0,APL011818576,20000000,2023-08-02 00:00:00,2023-07-22 06:00:55.607000,11,1100000,1210000,14222606
2,APL011825958,50000000,2023-08-14 00:00:00,2023-06-15 16:00:52.883000,59,2750000,3025000.0000000005,14169551
3,APL011828148,50000000,2023-08-18 00:00:00,2023-06-17 06:00:58.677000,62,2750000,3025000.0000000005,14173732


1.2. VINAPHONE

In [20]:

vnpt_lead_list = []
list_month = [month, last_month]
for months in list_month:
    QUERY_STRING=f'"[TS-LOTTE-AVAY] Leadgen Services Reconciliation for {months} (VNPT)" has:attachment'

    gmail_messages = gmail_repo.search_messages_by_query(query=QUERY_STRING)
    for message in gmail_messages:
        attachment_pair_id = gmail_repo.query_attachments_from_email_message(message=message)
        df = gmail_repo.read_excel_attachment_to_pandas(attachment_pair = attachment_pair_id[0],
                                                   password = 'lfvn2023', sheet_name = 'TS_noscore')
        df['month'] = months
        vnpt_lead_list.append(df)

QUERY_STRING=f'"[TS-LOTTE-AVAY] Leadgen Services Reconciliation for {months} (VNPT)" has:attachment'

gmail_messages = gmail_repo.search_messages_by_query(query=QUERY_STRING)
for message in gmail_messages:
    attachment_pair_id = gmail_repo.query_attachments_from_email_message(message=message)
    df = gmail_repo.read_excel_attachment_to_pandas(attachment_pair = attachment_pair_id[0],
                                                   password = 'lfvn2023', sheet_name = 'TS_ns')
    df['month'] = last_2month
    print(attachment_pair_id)
    vnpt_lead_list.append(df)
        
vnpt_lead = pd.concat(vnpt_lead_list)
vnpt_lead.head()

ValueError: Worksheet named 'TS_noscore' not found

In [11]:
vnpt_lead = vnpt_lead.rename(columns = {'Q': 'qualify', 'Mobile2': 'lead_id'})
vnpt_lead.groupby(['month', 'Telco', 'SubmitData']).agg({'lead_id': 'count'})

,,,lead_id
month,Telco,SubmitData,
2023-04,vinaphone,avay,1382
2023-05,vinaphone,avay,636
2023-06,vinaphone,avay,106


In [12]:
QUERY_STRING=f'"[TS-LOTTE-AVAY] Leadgen Services Reconciliation for {month} (VNPT)" has:attachment'

gmail_messages = gmail_repo.search_messages_by_query(query=QUERY_STRING)

for message in gmail_messages:
    attachment_pair_id = gmail_repo.query_attachments_from_email_message(message=message)
    vnpt_data_disbursed = gmail_repo.read_excel_attachment_to_pandas(attachment_pair = attachment_pair_id[0],
                                                   password = 'lfvn2023', sheet_name = 'E_TS vina no score', skiprows= 9, usecols=lambda x: 'Unnamed' not in x)

vnpt_data_disbursed.head()

,Application No,CUSTOMER_NAME,PHONE,NATIONAL_ID,LOANAMOUNT,DISBURSE_DATE,Import date,No of days,Fee non Vat,Fee VAT,Lead_ID


In [13]:
vnpt_data_disbursed['No of days'] = vnpt_data_disbursed['No of days'].astype(int)
vnpt_data_disbursed = vnpt_data_disbursed[vnpt_data_disbursed['No of days']<= 60]
vnpt_data_disbursed.rename(columns = {'Lead_ID':'lead_id', 'PHONE':'phone_number'}, inplace = True)
vnpt_data_disbursed.columns = vnpt_data_disbursed.columns.str.lower()
vnpt_data_disbursed.columns = vnpt_data_disbursed.columns.str.replace(' ', '_')
vnpt_data_disbursed

,application_no,customer_name,phone_number,national_id,loanamount,disburse_date,import_date,no_of_days,fee_non_vat,fee_vat,lead_id


1.3 VIETTEL

In [21]:
QUERY_STRING=f'"[TS-LOTTE-AVAY] Leadgen Services Reconciliation for {month}" NOT "[TS-LOTTE-AVAY] Leadgen Services Reconciliation for {month} (VNPT)" has:attachment'

gmail_messages = gmail_repo.search_messages_by_query(query=QUERY_STRING)

for message in gmail_messages:
    attachment_pair_id = gmail_repo.query_attachments_from_email_message(message=message)
    print(attachment_pair_id[0].attachment_file_name)
    vt_lead = gmail_repo.read_excel_attachment_to_pandas(attachment_pair = attachment_pair_id[0],
                                                   password = 'lfvn2023', sheet_name = 'Sheet1')
vt_lead

Q_2023.08_TS_viettel_avay_sent.xlsx


,Q,Reason,ImportDate,Mobile2,Address,Sex,DOB,Age,SourceData,Score,Telco,SubmitData,CIC,DuplicateData,LoanDemand,FailReason,CallBackReason,FollowReason
0,N,Không nghe máy,45167.66304398148,14224556,Hà Nội,NaN,1993-01-01,NaN,TrustingSocial,NaN,viettel,avay,NaN,NaN,NaN,NaN,NaN,NaN
1,N,Call back,45167.64306712963,14224555,Quảng Ninh,NaN,1997-01-01,NaN,TrustingSocial,NaN,viettel,avay,NaN,NaN,NaN,NaN,01 - KH bận,NaN
2,N,Nợ xấu,45167.59636574074,14224554,Đồng Tháp,NaN,1996-01-01,01/01/1996,TrustingSocial,NaN,viettel,avay,Nợ chú ý,NaN,NaN,08.1 - CIC nợ xấu,NaN,NaN
3,N,Call back,45167.47346064815,14224552,Bình Định,NaN,1995-01-01,NaN,TrustingSocial,NaN,viettel,avay,NaN,NaN,NaN,NaN,01 - KH bận,NaN
4,N,Hồ sơ chờ thu,45167.39784722222,14224550,Hà Nội,NaN,1977-01-01,17/10/1977,TrustingSocial,NaN,viettel,avay,CIC thỏa mãn,Không trùng data,08 - Có nhu cầu,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,N,Liên hệ quá 3 lần không được,45141.25068287037,14224092,Hà Nội,NaN,1993-01-01,01/01/1992,TrustingSocial,NaN,viettel,avay,CIC thỏa mãn,Không trùng data,NaN,02.2 - Giấy tờ không thỏa theo điều kiện sản phẩm,NaN,NaN
315,N,Liên hệ quá 3 lần không được,45141.25068287037,14224091,Hồ Chí Minh,Nam,1974-01-01,01/11/1974,TrustingSocial,NaN,viettel,avay,CIC thỏa mãn,Không trùng data,NaN,NaN,NaN,NaN
316,N,Call back,45141.25067129629,14224089,Tây Ninh,NaN,1997-01-01,NaN,TrustingSocial,NaN,viettel,avay,NaN,NaN,NaN,NaN,NaN,NaN
317,N,Nợ xấu,45141.25067129629,14224088,Hà Nội,NaN,1978-01-01,01/01/1978,TrustingSocial,NaN,viettel,avay,Nợ xấu,NaN,09 - Không có nhu cầu: Không muốn vay nữa,08.1 - CIC nợ xấu,NaN,NaN


In [22]:
vt_lead = vt_lead[vt_lead['Q'] == 'Y']
vt_lead = vt_lead.rename(columns = {'Q': 'qualify', 'Mobile2': 'lead_id'})
vt_lead.groupby([ 'Telco', 'SubmitData']).agg({'lead_id': 'count'})

,,lead_id
Telco,SubmitData,


In [23]:
lotte_mbf = mbf_lead[['lead_id', 'DuplicateData', 'qualify', 'Reason']]
lotte_avay = vt_lead[['lead_id', 'DuplicateData', 'qualify', 'Reason']]
# lotte_vina = vnpt_lead[['lead_id', 'DuplicateData', 'qualify', 'Reason']]

lotte_mbf['telco'] = 'mobifone'
lotte_mbf['channel'] = 'sms'

lotte_avay['telco'] = 'viettel'
lotte_avay['channel'] = 'avay'

# lotte_vina['telco'] = 'vinaphone'
# lotte_vina['channel'] = 'avay'

lotte_vina = pd.DataFrame()
vnpt_data_disbursed = pd.DataFrame()

/tmp/ipykernel_510939/3923656465.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lotte_mbf['telco'] = 'mobifone'
/tmp/ipykernel_510939/3923656465.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lotte_mbf['channel'] = 'sms'


In [24]:
lotte = pd.concat([lotte_mbf, lotte_avay, lotte_vina])
lotte_disbursed = pd.concat([vnpt_data_disbursed, mbf_data_disbursed])
lotte_disbursed['fee_non_vat'] = pd.to_numeric(lotte_disbursed['fee_non_vat'])
lotte_disbursed['fee_vat'] = pd.to_numeric(lotte_disbursed['fee_vat'])
lotte_disbursed['loanamount'] = pd.to_numeric(lotte_disbursed['loanamount'])
lotte_disbursed

,application_no,loanamount,disburse_date,import_date,no_of_days,fee_non_vat,fee_vat,lead_id
0,APL011818576,20000000,2023-08-02 00:00:00,2023-07-22 06:00:55.607000,11,1100000,1210000.0,14222606
2,APL011825958,50000000,2023-08-14 00:00:00,2023-06-15 16:00:52.883000,59,2750000,3025000.0,14169551
3,APL011828148,50000000,2023-08-18 00:00:00,2023-06-17 06:00:58.677000,62,2750000,3025000.0,14173732


In [25]:
lotte.loc[lotte['lead_id'].isin(lotte_disbursed['lead_id'])].reset_index()

,index,lead_id,DuplicateData,qualify,Reason,telco,channel
0,1020,14222606,Không trùng data,N,Hồ sơ chờ thu,mobifone,sms
1,1477,14169551,NaN,N,Hồ sơ chờ thu,mobifone,sms
2,1630,14173732,Không trùng data,N,Liên hệ quá 3 lần không được,mobifone,sms


In [26]:
#check duplicate lead_id
dup_id = lotte[lotte['lead_id'].duplicated(keep=False)]
create_or_update_sheet(wb = lotte_check_report, new_sheet = 'dup_id', input_df = dup_id)

2. QUERY DATA TS

In [27]:
lotte_query = f"""select id as lead_id, phone_number, bound_code, score, telco_code,
                    to_char(sent_at AT TIME ZONE 'UTC', 'YYYY-MM-DD') as sent_date,
                    sent_at,
                    (case when ((other ->> 'sender')::text = 'sms' OR (other ->> 'sender')::text is null and (other ->> 'channel')::text is null) then 'SMS' 
                    when (other ->> 'sender')::text = 'cold_call' then 'ColdCall'
                    when (other ->> 'channel')::text = 'sms' then 'SMS' 
                    when (other ->> 'channel')::text = 'avay_sms' then 'AVAY_SMS' 
                    when (other ->> 'sender')::text = 'avay' and (other ->> 'channel')::text is null then 'AVAY' 
                    when (other ->> 'sender')::text = 'viet_tin' and (other ->> 'channel')::text is null then 'VIET_TIN' else 'NOT_NULL' end) as channel,
                    (case when (pre_scoring_data ->> 'is_qualified')::boolean = true then 1 else 0 end) as qualified,
                    (other ->> 'id_card_number') as nid
                    from phone_infos 
                    where true
                    and bank_code like 'lotte'
                    and was_sent::boolean = true"""
ts_sent = pd.read_sql(lotte_query, conn)
ts_sent

/tmp/ipykernel_510939/2019333272.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ts_sent = pd.read_sql(lotte_query, conn)


,lead_id,phone_number,bound_code,score,telco_code,sent_date,sent_at,channel,qualified,nid
0,5114457,84932540793,LOTTE03,627,mobifone,2019-04-05,2019-04-05 06:30:14.415778,SMS,1,None
1,5915248,84965240203,LOTTE04,508,viettel,2019-08-12,2019-08-12 00:31:05.699609,AVAY,1,079082007284
2,7186198,84934807371,LOTTE01,644,mobifone,2020-03-27,2020-03-27 07:04:11.399777,SMS,1,025773717
3,7186227,84908331281,LOTTE01,649,mobifone,2020-03-27,2020-03-27 07:08:11.775764,SMS,1,025628842
4,7177273,84937726368,LOTTE01,640,mobifone,2020-03-25,2020-03-25 23:12:05.767167,SMS,1,331437704
...,...,...,...,...,...,...,...,...,...,...
372868,14224466,84965173243,LOTTE00,-1,viettel,2023-08-24,2023-08-24 06:54:01.766082,AVAY,0,091197011073
372869,14224469,841652618644,LOTTE00,-1,viettel,2023-08-24,2023-08-24 23:00:01.594116,AVAY,1,079197036349
372870,14224489,84978515742,LOTTE00,-1,viettel,2023-08-25,2023-08-25 23:00:01.653464,AVAY,1,083094012723
372871,14224491,84963123749,LOTTE00,-1,viettel,2023-08-25,2023-08-25 23:00:02.615113,AVAY,1,091186000811


In [28]:
ts_sent['bound_code'] = np.where((ts_sent['score'] == 650) & (ts_sent['telco_code'] == 'mobifone'), 'LOTTE02', ts_sent['bound_code'])
ts_sent['batch_month'] = ts_sent['sent_date'].str[:7]
ts_sent['sender'] = np.where(ts_sent['channel'] == 'AVAY', 'AVAY', np.where((ts_sent['channel'] == 'SMS') & (ts_sent['telco_code'] == 'viettel'), 'VIETTEL', np.where((ts_sent['channel'] == 'SMS') & (ts_sent['telco_code'] == 'mobifone'), 'MOBIFONE', 'other')))
ts_sent.head()

,lead_id,phone_number,bound_code,score,telco_code,sent_date,sent_at,channel,qualified,nid,batch_month,sender
0,5114457,84932540793,LOTTE03,627,mobifone,2019-04-05,2019-04-05 06:30:14.415778,SMS,1,None,2019-04,MOBIFONE
1,5915248,84965240203,LOTTE04,508,viettel,2019-08-12,2019-08-12 00:31:05.699609,AVAY,1,079082007284,2019-08,AVAY
2,7186198,84934807371,LOTTE01,644,mobifone,2020-03-27,2020-03-27 07:04:11.399777,SMS,1,025773717,2020-03,MOBIFONE
3,7186227,84908331281,LOTTE01,649,mobifone,2020-03-27,2020-03-27 07:08:11.775764,SMS,1,025628842,2020-03,MOBIFONE
4,7177273,84937726368,LOTTE01,640,mobifone,2020-03-25,2020-03-25 23:12:05.767167,SMS,1,331437704,2020-03,MOBIFONE


3. JOIN TS & LOTTE DATA

In [29]:
ts_sent['lead_id'] = ts_sent['lead_id'].astype(str)
lotte['lead_id'] = lotte['lead_id'].astype(str)
lotte_final = pd.merge(lotte, ts_sent, how='left', on = 'lead_id', suffixes = ['_ltfn', '_ts'])
lotte_final

,lead_id,DuplicateData,qualify,Reason,telco,channel_ltfn,phone_number,bound_code,score,telco_code,sent_date,sent_at,channel_ts,qualified,nid,batch_month,sender
0,14224015,Không trùng data,N,Follow,mobifone,sms,84903820865,LOTTE02,647,mobifone,2023-07-31,2023-07-31 23:00:03.547017,SMS,1,075195022719,2023-07,MOBIFONE
1,14224012,NaN,N,Follow,mobifone,sms,841267697386,LOTTE03,700,mobifone,2023-07-31,2023-07-31 23:00:03.131206,SMS,1,083194000648,2023-07,MOBIFONE
2,14224006,NaN,N,Nợ xấu,mobifone,sms,841265976352,LOTTE02,633,mobifone,2023-07-31,2023-07-31 23:00:02.703628,SMS,1,803093001237,2023-07,MOBIFONE
3,14215160,Không trùng data,N,Ngoài độ tuổi,mobifone,sms,84905838658,LOTTE03,686,mobifone,2023-06-30,2023-06-30 23:00:06.570785,SMS,1,048067001490,2023-06,MOBIFONE
4,14215183,Không trùng data,N,Hồ sơ chờ thu,mobifone,sms,841266897009,LOTTE02,649,mobifone,2023-06-30,2023-06-30 23:00:07.359252,SMS,1,079084016624,2023-06,MOBIFONE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3970,14215042,NaN,N,Call back,mobifone,sms,841227770307,LOTTE02,640,mobifone,2023-06-30,2023-06-30 09:30:07.022690,SMS,1,082196015785,2023-06,MOBIFONE
3971,14215054,Không trùng data,N,Nợ chú ý,mobifone,sms,84938437073,LOTTE02,635,mobifone,2023-06-30,2023-06-30 09:35:04.947154,SMS,1,046168000256,2023-06,MOBIFONE
3972,14215063,Không trùng data,N,Nợ xấu,mobifone,sms,841212341530,LOTTE03,650,mobifone,2023-06-30,2023-06-30 09:40:05.202318,SMS,1,075078015104,2023-06,MOBIFONE
3973,14215065,Không trùng data,N,Call back,mobifone,sms,84896689177,LOTTE03,673,mobifone,2023-06-30,2023-06-30 09:40:07.794623,SMS,1,201727280,2023-06,MOBIFONE


In [30]:
#check TS lead or not
not_ts_lead = lotte_final.loc[pd.isna(lotte_final['batch_month'])]
create_or_update_sheet(wb = lotte_check_report, new_sheet = 'not_ts_lead', input_df = not_ts_lead)

In [31]:
ts_sent.groupby(['batch_month','sender', 'telco_code', 'bound_code']).size().reset_index()

,batch_month,sender,telco_code,bound_code,0
0,2018-12,VIETTEL,viettel,LOTTE01,1719
1,2019-01,VIETTEL,viettel,LOTTE01,5377
2,2019-02,VIETTEL,viettel,LOTTE01,1212
3,2019-02,VIETTEL,viettel,LOTTE02,1250
4,2019-03,MOBIFONE,mobifone,LOTTE01,3092
...,...,...,...,...,...
263,2023-07,MOBIFONE,mobifone,LOTTE03,1083
264,2023-08,AVAY,viettel,LOTTE00,421
265,2023-08,AVAY,vinaphone,LOTTE00,72
266,2023-09,AVAY,viettel,LOTTE00,334


In [32]:
lotte_final['score'] = lotte_final['score'].astype(int)
lotte_final['score_group'] = np.where(lotte_final['bound_code'] == 'LOTTE00', 'No Score'
                                      , np.where(lotte_final['bound_code'] == 'LOTTE01', 'Medium'
                                                 , np.where(lotte_final['bound_code'] == 'LOTTE02', 'High score'
                                                            , np.where(lotte_final['bound_code'] == 'LOTTE03', 'Low', 'other'))))

lotte_final['score_range'] = np.where((lotte_final['sender'] == 'MOBIFONE') & (lotte_final['score'] >= 601) & (lotte_final['score'] <= 630), '[601-630]'
                                      , np.where((lotte_final['sender'] == 'MOBIFONE') & (lotte_final['score'] >= 631) & (lotte_final['score'] <= 649), '[631-649]'
                                                 , np.where((lotte_final['sender'] == 'MOBIFONE') & (lotte_final['score'] >= 650), '[650+]'
                                                            , np.where((lotte_final['sender'] == 'AVAY') & (lotte_final['score'] >= 581) & (lotte_final['score'] < 600), '[581-600]'
                                                                       , np.where((lotte_final['sender'] == 'AVAY') & (lotte_final['score'] >= 600) & (lotte_final['score'] < 630), '[600-629]'
                                                                                  , np.where((lotte_final['sender'] == 'AVAY') & (lotte_final['score'] >= 630), '[630+]'
                                                                                             , np.where((lotte_final['sender'] == 'AVAY') & (lotte_final['bound_code'] == 'LOTTE00'), 'no_score'
                                                                                                        , np.where((lotte_final['sender'] == 'VIETTEL') & (lotte_final['bound_code'] == 'LOTTE01'), '[600-629]'
                                                                                                                   , np.where((lotte_final['sender'] == 'VIETTEL') & (lotte_final['bound_code'] == 'LOTTE02'), '[630+]', 'other')))))))))

lotte_final.head()

,lead_id,DuplicateData,qualify,Reason,telco,channel_ltfn,phone_number,bound_code,score,telco_code,sent_date,sent_at,channel_ts,qualified,nid,batch_month,sender,score_group,score_range
0,14224015,Không trùng data,N,Follow,mobifone,sms,84903820865,LOTTE02,647,mobifone,2023-07-31,2023-07-31 23:00:03.547017,SMS,1,075195022719,2023-07,MOBIFONE,High score,[631-649]
1,14224012,NaN,N,Follow,mobifone,sms,841267697386,LOTTE03,700,mobifone,2023-07-31,2023-07-31 23:00:03.131206,SMS,1,083194000648,2023-07,MOBIFONE,Low,[650+]
2,14224006,NaN,N,Nợ xấu,mobifone,sms,841265976352,LOTTE02,633,mobifone,2023-07-31,2023-07-31 23:00:02.703628,SMS,1,803093001237,2023-07,MOBIFONE,High score,[631-649]
3,14215160,Không trùng data,N,Ngoài độ tuổi,mobifone,sms,84905838658,LOTTE03,686,mobifone,2023-06-30,2023-06-30 23:00:06.570785,SMS,1,048067001490,2023-06,MOBIFONE,Low,[650+]
4,14215183,Không trùng data,N,Hồ sơ chờ thu,mobifone,sms,841266897009,LOTTE02,649,mobifone,2023-06-30,2023-06-30 23:00:07.359252,SMS,1,079084016624,2023-06,MOBIFONE,High score,[631-649]


In [33]:
lotte_final.groupby(['batch_month', 'sender', 'telco_code', 'bound_code']).size().reset_index()

,batch_month,sender,telco_code,bound_code,0
0,2023-05,MOBIFONE,mobifone,LOTTE02,5
1,2023-05,MOBIFONE,mobifone,LOTTE03,28
2,2023-06,MOBIFONE,mobifone,LOTTE02,716
3,2023-06,MOBIFONE,mobifone,LOTTE03,2039
4,2023-07,MOBIFONE,mobifone,LOTTE02,331
5,2023-07,MOBIFONE,mobifone,LOTTE03,856


In [34]:
#check duplicate nid
leads_time_query = """select   (other ->> 'id_card_number') as nid
                       , count(*) as num_sent
                       , max(sent_at) as recent_sent, min(sent_at) as earliest_sent
                       , array_to_string(array_agg(distinct sent_at),',') as dates_sen
                       , (DATE_PART('year', max(sent_at)::date) - DATE_PART('year', min(sent_at)::date)) * 12 +
                         (DATE_PART('month', max(sent_at)::date) - DATE_PART('month', min(sent_at)::date)) as month_diff
              from     phone_infos as pi
              where    bank_code = 'lotte'
                       and was_sent::boolean = true
                       group by 1
                       having count(*) >1
             """
leads_time_sent = pd.read_sql(leads_time_query, conn)


/tmp/ipykernel_510939/3910054108.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  leads_time_sent = pd.read_sql(leads_time_query, conn)


In [35]:
dup_phone = lotte_final[lotte_final['phone_number'].duplicated(keep=False)]
create_or_update_sheet(wb = lotte_check_report, new_sheet = 'dup_phone', input_df = dup_phone)

In [36]:
dup_nid = lotte_final[lotte_final['nid'].duplicated(keep=False)]
create_or_update_sheet(wb = lotte_check_report, new_sheet = 'dup_nid', input_df = dup_nid)

In [37]:
# Check duplicated data: filter cases which lotte note as "trung data":
lotte_final[lotte_final['Reason'] == "Trùng data"].groupby('channel_ts').size().reset_index()

,channel_ts,0


In [38]:
lotte_final_1 = lotte_final.loc[lotte_final['lead_id'].isin(lotte_disbursed['lead_id']) | (lotte_final['telco'] == 'viettel')]
lotte_final_2 = pd.merge(lotte_final_1, lotte_disbursed, on='lead_id', how='left')

lotte_final_2.loc[lotte_final_2['telco'] == 'mobifone', 'qualify'] = 'Y'
lotte_final_2.loc[lotte_final_2['telco'] == 'mobifone', 'reason'] = 'Giải ngân thành công'
lotte_final_2.loc[lotte_final_2['telco'] == 'vinaphone', 'qualify'] = 'Y'
lotte_final_2.loc[lotte_final_2['telco'] == 'vinaphone', 'reason'] = 'Giải ngân thành công'

lotte_final_1.groupby(['telco_code', 'batch_month']).size().reset_index(name='count')


,telco_code,batch_month,count
0,mobifone,2023-06,2
1,mobifone,2023-07,1


In [39]:

lotte_final_2['correct_qualified'] = lotte_final_2['qualify']
lotte_final_2.groupby(['channel_ts', 'sender', 'score_range']).size().reset_index(name='count')

lotte_final_2['price'] = np.where(lotte_final_2['correct_qualified'] == 'Y'
                                  , np.where((lotte_final_2['channel_ts'] == 'SMS') & (lotte_final_2['sender'] == 'MOBIFONE'), lotte_final_2['fee_non_vat']
                                             , np.where((lotte_final_2['channel_ts'] == 'SMS') & (lotte_final_2['sender'] == 'VIETTEL') & (lotte_final_2['score_range'] == '[581-600]'), 140000
                                                        , np.where((lotte_final_2['channel_ts'] == 'SMS') & (lotte_final_2['sender'] == 'VIETTEL') & (lotte_final_2['score_range'] == '[600-629]'), 184000
                                                                   , np.where((lotte_final_2['channel_ts'] == 'SMS') & (lotte_final_2['sender'] == 'VIETTEL') & (lotte_final_2['score_range'] == '[630+]'), 280000
                                                                              , np.where((lotte_final_2['channel_ts'] == 'AVAY') & (lotte_final_2['sender'] == 'AVAY') & (lotte_final_2['score_range'] == '[581-600]'), 140000
                                                                                         , np.where((lotte_final_2['channel_ts'] == 'AVAY') & (lotte_final_2['sender'] == 'AVAY') & (lotte_final_2['score_range'] == '[600-629]'), 168000
                                                                                                    , np.where((lotte_final_2['channel_ts'] == 'AVAY') & (lotte_final_2['sender'] == 'AVAY') & (lotte_final_2['score_range'] == '[630+]'), 168000
                                                                                                               , np.where((lotte_final_2['channel_ts'] == 'AVAY') & (lotte_final_2['sender'] == 'AVAY') & (lotte_final_2['score_range'] == 'no_score'), lotte_final_2['fee_non_vat'], 0)))))))), 0)
lotte_final_2['telco'] = np.where(lotte_final_2['bound_code'].isin(['LOTTE01', 'LOTTE02', 'LOTTE03']), lotte_final_2['telco_code'], 'Others')

lotte_final_2.loc[lotte_final_2['qualify'] == 'Y'].groupby(['telco_code', 'channel_ts', 'score_range']).agg({'price': 'sum', 'lead_id': 'count'}).reset_index()

,telco_code,channel_ts,score_range,price,lead_id
0,mobifone,SMS,[650+],6600000,3


In [40]:
lotte_final_2

,lead_id,DuplicateData,qualify,Reason,telco,channel_ltfn,phone_number,bound_code,score,telco_code,sent_date,sent_at,channel_ts,qualified,nid,batch_month,sender,score_group,score_range,application_no,loanamount,disburse_date,import_date,no_of_days,fee_non_vat,fee_vat,reason,correct_qualified,price
0,14222606,Không trùng data,Y,Hồ sơ chờ thu,mobifone,sms,84939190955,LOTTE03,650,mobifone,2023-07-21,2023-07-21 23:00:02.044282,SMS,1,093083010048,2023-07,MOBIFONE,Low,[650+],APL011818576,20000000,2023-08-02 00:00:00,2023-07-22 06:00:55.607000,11,1100000,1210000.0,Giải ngân thành công,Y,1100000
1,14169551,NaN,Y,Hồ sơ chờ thu,mobifone,sms,84908583330,LOTTE03,710,mobifone,2023-06-15,2023-06-15 09:00:05.625503,SMS,1,079083019379,2023-06,MOBIFONE,Low,[650+],APL011825958,50000000,2023-08-14 00:00:00,2023-06-15 16:00:52.883000,59,2750000,3025000.0,Giải ngân thành công,Y,2750000
2,14173732,Không trùng data,Y,Liên hệ quá 3 lần không được,mobifone,sms,84939060391,LOTTE03,679,mobifone,2023-06-16,2023-06-16 23:00:11.155100,SMS,1,079093006651,2023-06,MOBIFONE,Low,[650+],APL011828148,50000000,2023-08-18 00:00:00,2023-06-17 06:00:58.677000,62,2750000,3025000.0,Giải ngân thành công,Y,2750000


In [41]:
summary_total = lotte_final_2.groupby('score_range').agg({'lead_id': 'count', 'correct_qualified': lambda x: sum(x == 'Y')
                                                          , 'loanamount': 'sum', 'price': 'sum'}).rename(columns={'lead_id': 'Total_lead', 'correct_qualified': 'Qualified_leads', 'price': 'Fee_non_VAT'}).reset_index()
summary_total['Fee_icd_VAT'] = summary_total['Fee_non_VAT'] * 1.1
summary_total = summary_total.append(summary_total.agg(['sum']))


summary_avay_vt = lotte_final_2.loc[(lotte_final_2['channel_ts'] == 'AVAY') & (lotte_final_2['telco_code'] == 'viettel')
                                    ].groupby('score_range').agg({'lead_id': 'count', 'correct_qualified': lambda x: sum(x == 'Y')
                                                                  , 'price': 'sum'}).rename(columns={'lead_id': 'Total_lead', 'correct_qualified': 'qualified_leads', 'price': 'fee_non_vat'}).reset_index()
summary_avay_vt['fee_w_vat'] = summary_avay_vt['fee_non_vat'] * 1.1
summary_avay_vt = summary_avay_vt.append(summary_avay_vt.agg(['sum']))

summary_avay_vn = lotte_final_2.loc[(lotte_final_2['channel_ts'] == 'AVAY') & (lotte_final_2['telco_code'] == 'vinaphone')].groupby('score_range').agg({'correct_qualified': lambda x: sum(x == 'Y'), 'loanamount': 'sum', 'price': 'sum'}).rename(columns={'correct_qualified': 'disbursed_leads', 'price': 'fee_non_vat'}).reset_index()
summary_avay_vn['fee_w_vat'] = summary_avay_vn['fee_non_vat'] * 1.1
summary_avay_vn = summary_avay_vn.append(summary_avay_vn.agg(['sum']))

summary_mobi = lotte_final_2.loc[(lotte_final_2['channel_ts'] != 'AVAY') & (lotte_final_2['telco_code'] == 'mobifone')].groupby('batch_month').agg({'correct_qualified': lambda x: sum(x == 'Y'), 'loanamount': 'sum', 'price': 'sum'}).rename(columns={'correct_qualified': 'disbursed_leads', 'price': 'fee_non_vat'}).reset_index()
summary_mobi['fee_w_vat'] = summary_mobi['fee_non_vat'] * 1.1
summary_mobi = summary_mobi.append(summary_mobi.agg(['sum']))
summary_mobi

# summary_viettel = lotte_final_2[(lotte_final_2['channel_ts'] != 'AVAY') & (lotte_final_2['telco_code'] == 'viettel')].groupby('score_range').agg({'lead_id': 'count', 'correct_qualified': lambda x: sum(x == 'Y'), 'price': 'sum'}).rename(columns={'lead_id': 'Total_lead', 'correct_qualified': 'qualified_leads', 'price': 'fee_non_vat'})
# summary_viettel['fee_w_vat'] = summary_viettel['fee_non_vat'] * 1.1
# summary_viettel = summary_viettel.append(summary_viettel.agg(['sum']))
# summary_viettel

/tmp/ipykernel_510939/2137283054.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_total = summary_total.append(summary_total.agg(['sum']))
/tmp/ipykernel_510939/2137283054.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_avay_vt = summary_avay_vt.append(summary_avay_vt.agg(['sum']))
/tmp/ipykernel_510939/2137283054.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_avay_vn = summary_avay_vn.append(summary_avay_vn.agg(['sum']))
/tmp/ipykernel_510939/2137283054.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_mobi = summary_mobi.append(summary_mobi.agg(['sum']))


,batch_month,disbursed_leads,loanamount,fee_non_vat,fee_w_vat
0,2023-06,2,100000000,5500000,6050000.0
1,2023-07,1,20000000,1100000,1210000.0
sum,2023-062023-07,3,120000000,6600000,7260000.0


In [42]:
create_or_update_sheet(wb = lotte_summary, new_sheet = 'summary', input_df = summary_total)

create_or_update_sheet(wb = lotte_summary, new_sheet = 'summary_avay_vt', input_df = summary_avay_vt)

create_or_update_sheet(wb = lotte_summary, new_sheet = 'summary_avay_vn', input_df = summary_avay_vn)

create_or_update_sheet(wb = lotte_summary, new_sheet = 'summary_mobi', input_df = summary_mobi)

In [43]:
#add lead source
avay_bq_acc = AvayBQServiceAcc()

avay_df = avay_bq_acc.client.query(f"""
select case when source = 'prod.sendo' then 'Sendo'
        when source = 'prod.chotot' then 'ChoTot'
        when source = 'prod.kalapa' then 'Kalapa'
        when source = 'prod.vpp.viettel' then 'Viettel Pay Pro'
        when source = 'prod.vtp.viettel' then 'Viettel Pay'
        when utm_source like 'dir%' then utm_source
        when source = 'vaycucde.vn' then 'Vaycucde'
        when utm_source = 'fimar' then 'Fimar'
        when utm_source like '%ccesstra%' then 'Accesstrade'
        when utm_source = 'masoffer' then 'Masoffer'
        when utm_source = 'vaysieude' then 'Vaysieude'
        when utm_source = 'thomaytaichinh' then 'Thomaytaichinh'
        when utm_source = 'adpia' THEN 'Adpia'
        when utm_source like '%acebook%' or utm_source like '%fb%' or utm_source like 'chatfuel%' then 'Facebook'
        when source = 'android_app' then 'Android App'
        when utm_campaign = 'adtima' or utm_source like 'zal%' then 'Zalo'
        when utm_source = 'sales_doubler' then 'Sale Doubler'
        when source = 'prod.zalo' then 'Zalo CC'
        when utm_source like 'google_ad%' then 'Google ads'
        when utm_source = 'zns' then 'ZNS'
        when utm_source = 'mpcc' then 'mpcc'
        when utm_source like 'mgi%' then 'MGID'
        when utm_source = 'fingo' then 'Fingo'
        when utm_source = 'esmart' then 'eSmart'
        when utm_source = 'linh_dan_spa' then 'Linh Dan Spa'
        when utm_source = 'cafe_anh' then 'Cafe Anh'
        when utm_source = 'tanvd' then 'Tanvd'
        when utm_source = 'adfly' then 'Adfly'
        when utm_source = 'coccoc' then 'Coccoc'
        when utm_source in ('adskeeper','adskeeper.com') then 'Adskeeper'
        when utm_source = 'vinfin' then 'Vinfin'
        when utm_source = 'propellerads' then 'Propellerads'
        when utm_source = 'dinos' then 'Dinos'
        when utm_source = 'clickadu' then 'Clickadu'
        else 'Others' end as lead_source
    ,lead_id
  from `avay-a9925.dwh.loans` l
  left join `avay-a9925.dwh.registrations` r on r.id = l.reg_id
  left join `avay-a9925.dwh.otps` o on r.otp_id = o.id
  where l.lead_id is not null
                          ;""").result().to_arrow().to_pandas()
  
avay_df

,lead_source,lead_id
0,Coccoc,11649210
1,Zalo,9580349
2,Vaycucde,8482734
3,Zalo CC,9203748
4,Adfly,8168109
...,...,...
3652155,Viettel Pay,8212709
3652156,Viettel Pay,8190783
3652157,Viettel Pay,8190788
3652158,Viettel Pay,13751551


In [44]:
avay_df['lead_id'] = avay_df['lead_id'].astype(str)
lotte_final_add_source = pd.merge(lotte_final_2, avay_df, how='left', on='lead_id')
lotte_final_add_source

,lead_id,DuplicateData,qualify,Reason,telco,channel_ltfn,phone_number,bound_code,score,telco_code,sent_date,sent_at,channel_ts,qualified,nid,batch_month,sender,score_group,score_range,application_no,loanamount,disburse_date,import_date,no_of_days,fee_non_vat,fee_vat,reason,correct_qualified,price,lead_source
0,14222606,Không trùng data,Y,Hồ sơ chờ thu,mobifone,sms,84939190955,LOTTE03,650,mobifone,2023-07-21,2023-07-21 23:00:02.044282,SMS,1,093083010048,2023-07,MOBIFONE,Low,[650+],APL011818576,20000000,2023-08-02 00:00:00,2023-07-22 06:00:55.607000,11,1100000,1210000.0,Giải ngân thành công,Y,1100000,NaN
1,14169551,NaN,Y,Hồ sơ chờ thu,mobifone,sms,84908583330,LOTTE03,710,mobifone,2023-06-15,2023-06-15 09:00:05.625503,SMS,1,079083019379,2023-06,MOBIFONE,Low,[650+],APL011825958,50000000,2023-08-14 00:00:00,2023-06-15 16:00:52.883000,59,2750000,3025000.0,Giải ngân thành công,Y,2750000,NaN
2,14173732,Không trùng data,Y,Liên hệ quá 3 lần không được,mobifone,sms,84939060391,LOTTE03,679,mobifone,2023-06-16,2023-06-16 23:00:11.155100,SMS,1,079093006651,2023-06,MOBIFONE,Low,[650+],APL011828148,50000000,2023-08-18 00:00:00,2023-06-17 06:00:58.677000,62,2750000,3025000.0,Giải ngân thành công,Y,2750000,NaN


In [45]:
to_write_disbursed = lotte_final_add_source.loc[lotte_final_2['correct_qualified'] == 'Y'].rename(
    columns={'sent_date': 'batch_date', 'price': 'commission', 'bound_code': 'product_code', 'disburse_date': 'disbursed_date', 
             'loanamount': 'loan_amount', 'channel_ts':'channel'}).assign(
        bank_code='lotte', 
        tenor=pd.NA, 
        fee_rate=pd.Series(np.where(lotte_final_2['telco_code'] == 'mobifone', 0.055, np.where(lotte_final_2['telco_code'] == 'vinaphone', 0.035, np.nan))), 
        disbursed_status='reconciled', 
        update_at=pd.Timestamp.now(), 
        other=pd.NA,
        ).loc[:, ['bank_code', 'lead_id', 'phone_number', 'batch_date', 'channel', 'product_code', 'nid', 
                               'telco_code', 'tenor', 'loan_amount', 'disbursed_date', 'fee_rate', 'commission','disbursed_status','update_at', 'lead_source']]

to_write_disbursed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 0 to 2
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   bank_code         3 non-null      object        
 1   lead_id           3 non-null      object        
 2   phone_number      3 non-null      object        
 3   batch_date        3 non-null      object        
 4   channel           3 non-null      object        
 5   product_code      3 non-null      object        
 6   nid               3 non-null      object        
 7   telco_code        3 non-null      object        
 8   tenor             0 non-null      object        
 9   loan_amount       3 non-null      int64         
 10  disbursed_date    3 non-null      object        
 11  fee_rate          3 non-null      float64       
 12  commission        3 non-null      int64         
 13  disbursed_status  3 non-null      object        
 14  update_at         3 non-null  

In [46]:
file_disbursed_stored = f'{path}{month}/lotte_disbursed_{month}.csv'

to_write_disbursed.to_csv(file_disbursed_stored, index=False)

file_path: Path = Path(file_disbursed_stored)

# Send success message to channel
clientSlack.chat_postMessage(**{"text": f"<@{user_id}> update disbursals Lotte {recon_month}"} , channel=channel_id)

# Upload file to channel
filetype = file_path.suffix[1:]
filename = file_path.name
with open(file_path, "rb") as file:
	clientSlack.files_upload(channels=channel_id, file=file, title=filename, filetype=filetype)

/home/linhnguyen/.local/lib/python3.9/site-packages/slack_sdk/web/internal_utils.py:450: UserWarning: client.files_upload() may cause some issues like timeouts for relatively large files. Our latest recommendation is to use client.files_upload_v2(), which is mostly compatible and much stabler, instead.
  warnings.warn(message)


In [47]:
print_df = lotte_final_2.rename(
    columns={'disburse_date': 'disbursed_date', 'loanamount': 'loan_amount', 'fee_non_vat':'fee_no_vat', 'fee_vat': 'fee_with_vat'}
    ).loc[:, ['lead_id', 'sent_date', 'channel_ts', 'telco_code', 'sender', 'score_range', 'loan_amount', 'disbursed_date', 'price']]

detail_mbf = print_df.loc[print_df['sender'] == 'MOBIFONE'].loc[:, ['lead_id', 'sent_date', 'score_range', 'loan_amount', 'disbursed_date', 'price']]
detail_avay_vt = print_df.loc[(print_df['sender'] == 'AVAY') & (print_df['telco_code'] == 'viettel')].loc[:, ['lead_id', 'sent_date', 'score_range', 'price']]
detail_avay_vn = print_df.loc[(print_df['sender'] == 'AVAY') & (print_df['telco_code'] == 'vinaphone')].loc[:, ['lead_id', 'sent_date', 'score_range', 'loan_amount', 'disbursed_date', 'price']]

In [48]:
create_or_update_sheet(wb = Lotte_detail_leads_by_sender, new_sheet = 'detail', input_df = print_df)
create_or_update_sheet(wb = lotte_statement, new_sheet = 'detail_mbf', input_df = detail_mbf)
create_or_update_sheet(wb = lotte_statement, new_sheet = 'detail_avay_vt', input_df = detail_avay_vt)
create_or_update_sheet(wb = lotte_statement, new_sheet = 'detail_avay_vn', input_df = detail_avay_vn)

mbf_disbursed_stored = f'{path}{month}/lotte_mbf_disbursed_{month}.csv'
avay_vt_disbursed_stored = f'{path}{month}/lotte_avay_vt_disbursed_{month}.csv'
avay_vn_disbursed_stored = f'{path}{month}/lotte_avay_vn_disbursed_{month}.csv'

detail_mbf.to_csv(mbf_disbursed_stored, index=False)
detail_avay_vt.to_csv(avay_vt_disbursed_stored, index=False)
detail_avay_vn.to_csv(avay_vn_disbursed_stored, index=False)

In [56]:
lotte_mobi_report = lotte_final_2[lotte_final_2['telco'] == 'mobifone']
lotte_mobi_report['change_qualified_reason'] = 'unchanged'
lotte_mobi_report['old_qualified'] = None
lotte_mobi_report['qualify'] = np.where(lotte_mobi_report['qualify']  == 'Y', 1
                                        ,np.where(lotte_mobi_report['qualify']  == 'N', 0,2))
lotte_mobi_report['report_date'] = end_date
lotte_mobi_report['qualified_date'] = end_date
lotte_mobi_report['telco_source'] = lotte_mobi_report['telco']
lotte_mobi_report['received_date'] = pd.to_datetime(lotte_mobi_report['sent_date']).dt.date
lotte_mobi_report['loan_amount'] = lotte_mobi_report['loanamount'].astype(int)
lotte_mobi_report['disbursed_date'] = pd.to_datetime(lotte_mobi_report['disburse_date']).dt.date
lotte_mobi_report['phone_number'] = lotte_mobi_report['lead_id']
lotte_mobi_report.head()

,lead_id,DuplicateData,qualify,Reason,telco,channel_ltfn,phone_number,bound_code,score,telco_code,sent_date,sent_at,channel_ts,qualified,nid,batch_month,sender,score_group,score_range,application_no,loanamount,disburse_date,import_date,no_of_days,fee_non_vat,fee_vat,reason,correct_qualified,price,change_qualified_reason,old_qualified,report_date,qualified_date,telco_source,received_date,loan_amount,disbursed_date
0,14222606,Không trùng data,1,Hồ sơ chờ thu,mobifone,sms,14222606,LOTTE03,650,mobifone,2023-07-21,2023-07-21 23:00:02.044282,SMS,1,093083010048,2023-07,MOBIFONE,Low,[650+],APL011818576,20000000,2023-08-02 00:00:00,2023-07-22 06:00:55.607000,11,1100000,1210000.0,Giải ngân thành công,Y,1100000,unchanged,None,2023-08-31,2023-08-31,mobifone,2023-07-21,20000000,2023-08-02
1,14169551,NaN,1,Hồ sơ chờ thu,mobifone,sms,14169551,LOTTE03,710,mobifone,2023-06-15,2023-06-15 09:00:05.625503,SMS,1,079083019379,2023-06,MOBIFONE,Low,[650+],APL011825958,50000000,2023-08-14 00:00:00,2023-06-15 16:00:52.883000,59,2750000,3025000.0,Giải ngân thành công,Y,2750000,unchanged,None,2023-08-31,2023-08-31,mobifone,2023-06-15,50000000,2023-08-14
2,14173732,Không trùng data,1,Liên hệ quá 3 lần không được,mobifone,sms,14173732,LOTTE03,679,mobifone,2023-06-16,2023-06-16 23:00:11.155100,SMS,1,079093006651,2023-06,MOBIFONE,Low,[650+],APL011828148,50000000,2023-08-18 00:00:00,2023-06-17 06:00:58.677000,62,2750000,3025000.0,Giải ngân thành công,Y,2750000,unchanged,None,2023-08-31,2023-08-31,mobifone,2023-06-16,50000000,2023-08-18


In [57]:
lotte_mobi_report_20230831 = lotte_mobi_report[['report_date', 'phone_number', 'received_date', 'score_range', 'telco_source', 'qualified_date', 'qualified', 'old_qualified', 'reason', 'change_qualified_reason', 'lead_id', 'loan_amount', 'disbursed_date']]
lotte_mobi_report_20230831.head()

,report_date,phone_number,received_date,score_range,telco_source,qualified_date,qualified,old_qualified,reason,change_qualified_reason,lead_id,loan_amount,disbursed_date
0,2023-08-31,14222606,2023-07-21,[650+],mobifone,2023-08-31,1,None,Giải ngân thành công,unchanged,14222606,20000000,2023-08-02
1,2023-08-31,14169551,2023-06-15,[650+],mobifone,2023-08-31,1,None,Giải ngân thành công,unchanged,14169551,50000000,2023-08-14
2,2023-08-31,14173732,2023-06-16,[650+],mobifone,2023-08-31,1,None,Giải ngân thành công,unchanged,14173732,50000000,2023-08-18


In [58]:
lotte_mobi_report_20230831.to_csv(f'{path}{month}/lotte_mobi_report_20230831.csv', index = False)

In [53]:
# drive_repo.upload_file_to_drive(file_path= Path(file_disbursed_stored), parent_folder_id= folder_month)
# drive_repo.upload_file_to_drive(file_path= Path('/home/linhnguyen/04.Reconcile/03.Lotte/Rmd/lotte_avay_VT_recon_detail_2023.pdf'), parent_folder_id= folder_month)
drive_repo.upload_file_to_drive(file_path= Path('/home/linhnguyen/04.Reconcile/03.Lotte/Rmd/lotte_MBF_recon_detail_2023.pdf'), parent_folder_id= folder_month)
drive_repo.upload_file_to_drive(file_path= Path('/home/linhnguyen/04.Reconcile/03.Lotte/2023-08/lotte_disbursed_2023-08.csv'), parent_folder_id= folder_month)

'1tYr9Vwb88fcFf4XZqMhpkWcagpWPwhHQ'